In [0]:
from PIL import Image
import numpy as np 
import os
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
# Caminho dos diretórios com as imagens
com_carie_dir = "/Volumes/workspace/tcc-panoramic-dental-dataset/radiografias-teste-10/com_carie/"
sem_carie_dir = "/Volumes/workspace/tcc-panoramic-dental-dataset/radiografias-teste-10/sem_carie/"

In [0]:
# Diretório de saída para as imagens aumentadas no Volume
OUTPUT_AUGMENTED_CARIES_DIR = "/Volumes/workspace/tcc-panoramic-dental-dataset/radiografias-teste-10/com_carie_augmented/"

# Diretório de saída se ele não existir
os.makedirs(OUTPUT_AUGMENTED_CARIES_DIR, exist_ok=True)
print(f"Diretório para imagens aumentadas: {OUTPUT_AUGMENTED_CARIES_DIR}")

In [0]:
# Lista de arquivos de imagem
com_carie_files = [f for f in os.listdir(com_carie_dir) if f.endswith('.png')]
sem_carie_files = [f for f in os.listdir(sem_carie_dir) if f.endswith('.png')]

print("\n--- Quantidade de arquivos ---")
print(f"Total de arquivos com cárie: {len(com_carie_files)}")
print(f"Total de arquivos sem cárie: {len(sem_carie_files)}")

In [0]:
# Função para carregar e redimensionar as imagens em memória
def load_and_resize_images(image_paths, target_size=(128, 128)):
    images = []
    for image_path in image_paths:
        try:
            img = Image.open(image_path).convert('RGB') # Garante 3 canais, se a imagem for P/B
            img_resized = img.resize(target_size)
            images.append(np.array(img_resized))  # Converter para array NumPy
        except FileNotFoundError:
            print(f"Aviso: Arquivo não encontrado - {image_path}. Ignorando.")
        except Exception as e:
            print(f"Erro ao carregar ou redimensionar {image_path}: {e}")
    return np.array(images)

In [0]:
# Carregar e redimensionar as imagens
images_com_carie = load_and_resize_images([os.path.join(com_carie_dir, f) for f in com_carie_files])
images_sem_carie = load_and_resize_images([os.path.join(sem_carie_dir, f) for f in sem_carie_files])

In [0]:
# Mostrar o shape das imagens carregadas e redimensionadas
print(f"Shape das imagens com cárie: {images_com_carie.shape}")
print(f"Shape das imagens sem cárie: {images_sem_carie.shape}")

In [0]:
# Função para Normalização das imagens
def normalize_images(images):
    # Divide os valores de pixel por 255 para normalizar para o intervalo [0, 1]
    return images / 255.0

In [0]:
# Normalizando as imagens de treinamento
images_com_carie = normalize_images(images_com_carie)
images_sem_carie = normalize_images(images_sem_carie)

In [0]:
# Verificando as shapes após normalização
print(f"Shape das imagens com cárie após normalização: {images_com_carie.shape}")
print(f"Shape das imagens sem cárie após normalização: {images_sem_carie.shape}")

In [0]:
# Salvar as imagens normalizadas em memória para o passo seguinte
X_train_original = np.concatenate((images_com_carie, images_sem_carie), axis=0)
y_train_original = np.concatenate((np.ones(len(images_com_carie)), np.zeros(len(images_sem_carie))), axis=0)

In [0]:
print(f"X_train_original shape: {X_train_original.shape}")
print(f"y_train_original shape: {y_train_original.shape}")

In [0]:
# Inicializando o gerador de Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,       # Rotação aleatória entre -20 e 20 graus
    width_shift_range=0.2,   # Deslocamento horizontal de até 20%
    height_shift_range=0.2,  # Deslocamento vertical de até 20%
    shear_range=0.2,         # Cisalhamento
    zoom_range=0.2,          # Zoom aleatório
    horizontal_flip=True,    # Flip horizontal
    fill_mode='nearest'      # Preenchimento de pixels ao transformar a imagem
)

In [0]:
# Definir número de imagens que queremos gerar (equalizando com o número de "sem cárie")
num_images_to_generate = len(images_sem_carie) - len(images_com_carie)

print(f"Número de imagens adicionais 'com cárie' a serem geradas: {num_images_to_generate}")

In [0]:
# Gerando e salvando as imagens aumentadas diretamente no Volume do Databricks
print("Iniciando geração e salvamento de imagens aumentadas...")
images_generated_count = 0

# O loop abaixo itera sobre cada imagem individualmente para gerar aumentações dela.
for image_idx, image in enumerate(images_com_carie):
    if images_generated_count >= num_images_to_generate:
        break # Para de gerar se o objetivo de imagens foi atingido

    # Reshape para (1, altura, largura, canais) para o datagen.flow de uma única imagem
    img_for_flow = np.expand_dims(image, axis=0)

    # Gerando as imagens com augmentation e SALVANDO DIRETAMENTE no Volume do Databricks
    for batch in datagen.flow(img_for_flow, batch_size=1,
                              save_to_dir=OUTPUT_AUGMENTED_CARIES_DIR,
                              save_prefix='com_carie_aug',
                              save_format='png'):
        images_generated_count += 1
        if images_generated_count % 100 == 0:
            print(f"Geradas e salvas: {images_generated_count} imagens...")
        if images_generated_count >= num_images_to_generate:
            break

print(f"\n✅ Total de imagens com cárie aumentadas geradas e salvas no disco: {images_generated_count}")

In [0]:
# Diretório temporário para o treinamento que unifique as pastas
TARGET_TRAIN_DIR = "/Volumes/workspace/tcc-panoramic-dental-dataset/dataset-treinamento-unificado/"
TARGET_TRAIN_COM_CARIE_DIR = os.path.join(TARGET_TRAIN_DIR, "com_carie")
TARGET_TRAIN_SEM_CARIE_DIR = os.path.join(TARGET_TRAIN_DIR, "sem_carie")

os.makedirs(TARGET_TRAIN_COM_CARIE_DIR, exist_ok=True)
os.makedirs(TARGET_TRAIN_SEM_CARIE_DIR, exist_ok=True)

# Copiar imagens originais 'com_carie' para a pasta unificada
print("Copiando imagens 'com_carie' originais para o diretório unificado...")
for f in com_carie_files:
    # Usar 'cp -f' para forçar a cópia se o arquivo já existir
    os.system(f"cp -f {os.path.join(com_carie_dir, f)} {os.path.join(TARGET_TRAIN_COM_CARIE_DIR, f)}")

In [0]:
# Copiar imagens aumentadas 'com_carie_augmented' para a pasta unificada
print("Copiando imagens 'com_carie' aumentadas para o diretório unificado...")
augmented_files = [f for f in os.listdir(OUTPUT_AUGMENTED_CARIES_DIR) if f.startswith('com_carie_aug')]
for f in augmented_files:
    os.system(f"cp -f {os.path.join(OUTPUT_AUGMENTED_CARIES_DIR, f)} {os.path.join(TARGET_TRAIN_COM_CARIE_DIR, f)}")

# Copiar imagens 'sem_carie' originais para a pasta unificada
print("Copiando imagens 'sem_carie' originais para o diretório unificado...")
for f in sem_carie_files:
    os.system(f"cp -f {os.path.join(sem_carie_dir, f)} {os.path.join(TARGET_TRAIN_SEM_CARIE_DIR, f)}")

print("\nEstrutura de diretórios para treinamento pronta.")

In [0]:
# Verificar o número final de imagens nas pastas
print(f"Total de imagens 'com_carie' no diretório unificado: {len(os.listdir(TARGET_TRAIN_COM_CARIE_DIR))}")
print(f"Total de imagens 'sem_carie' no diretório unificado: {len(os.listdir(TARGET_TRAIN_SEM_CARIE_DIR))}")